In [ ]:
import pandas as pd
from scapy.all import *
import numpy as np
import math
import os

#### Filtering UDP/QUIC packets

In [ ]:
def pcap_filter(path):
    try:
        os.system('tcpdump -r  '+ str(path)+'.pcap'+' -w ' + str(path) + '-0.pcap '+ 'udp')
        os.system('tcpdump -r  '+ str(path)+'-0.pcap'+' -w ' + str(path) + '-1.pcap '+ 'host 10.0.0.1')
        #os.system('rm -r  '+ str(path)+'.pcap')
        print('ok')
        
    except:
        print(path)
        pass

#### QoS features from UDP/QUIC packets and flows

In [ ]:
def pcap_processing(path):

    try:
        # dataframe declare with all columns name  # throughput(bps), packet size(bytes), inter-arrival time(second)
        data = pd.DataFrame(columns=['TP_D_avg', 'TP_D_max', 'TP_D_min', 'TP_D_medn', 'TP_D_std',
                                     'TP_D_10p', 'TP_D_20p', 'TP_D_30p', 'TP_D_40p', 'TP_D_50p', 'TP_D_60p', 'TP_D_70p', 'TP_D_80p', 'TP_D_90p',
                                     'TP_D_F25_avg', 'TP_D_F50_avg', 'TP_D_L25_avg', 'TP_D_L50_avg',

                                     'TP_U_avg', 'TP_U_max', 'TP_U_min', 'TP_U_medn', 'TP_U_std',
                                     'TP_U_10p', 'TP_U_20p', 'TP_U_30p', 'TP_U_40p', 'TP_U_50p', 'TP_U_60p', 'TP_U_70p', 'TP_U_80p', 'TP_U_90p',
                                     'TP_U_F25_avg', 'TP_U_F50_avg', 'TP_U_L25_avg', 'TP_U_L50_avg',

                                     'PC_D_total', 'PC_D_avg', 'PC_D_max', 'PC_D_min', 'PC_D_medn', 'PC_D_std',
                                     'PC_D_10p', 'PC_D_20p', 'PC_D_30p', 'PC_D_40p', 'PC_D_50p', 'PC_D_60p', 'PC_D_70p', 'PC_D_80p', 'PC_D_90p',
                                     'PC_D_F25_total', 'TP_D_F50_total', 'PC_D_L25_total', 'PC_D_L50_total',

                                     'PC_U_total', 'PC_U_avg', 'PC_U_max', 'PC_U_min', 'PC_U_medn', 'PC_U_std',
                                     'PC_U_10p', 'PC_U_20p', 'PC_U_30p', 'PC_U_40p', 'PC_U_50p', 'PC_U_60p', 'PC_U_70p', 'PC_U_80p', 'PC_U_90p',
                                     'PC_U_F25_total', 'TP_U_F50_total', 'PC_U_L25_total', 'PC_U_L50_total',

                                     'PS_D_avg', 'PS_D_max', 'PS_D_min', 'PS_D_medn', 'PS_D_std',
                                     'PS_D_10p', 'PS_D_20p', 'PS_D_30p', 'PS_D_40p', 'PS_D_50p', 'PS_D_60p', 'PS_D_70p', 'PS_D_80p', 'PS_D_90p',
                                     'PS_D_F25_avg', 'PS_D_F50_avg', 'PS_D_L25_avg', 'PS_D_L50_avg',

                                     'PS_U_avg', 'PS_U_max', 'PS_U_min', 'PS_U_medn', 'PS_U_std',
                                     'PS_U_10p', 'PS_U_20p', 'PS_U_30p', 'PS_U_40p', 'PS_U_50p', 'PS_U_60p', 'PS_U_70p', 'PS_U_80p', 'PS_U_90p',
                                     'PS_U_F25_avg', 'PS_U_F50_avg', 'PS_U_L25_avg', 'PS_U_L50_avg',

                                     'IAT_D_avg', 'IAT_D_max', 'IAT_D_min', 'IAT_D_medn', 'IAT_D_std',
                                     'IAT_D_10p', 'IAT_D_20p', 'IAT_D_30p', 'IAT_D_40p', 'IAT_D_50p', 'IAT_D_60p', 'IAT_D_70p', 'IAT_D_80p', 'IAT_D_90p',
                                     'IAT_D_F25_avg', 'IAT_D_F50_avg', 'IAT_D_L25_avg', 'IAT_D_L50_avg',

                                     'IAT_U_avg', 'IAT_U_max', 'IAT_U_min', 'IAT_U_medn', 'IAT_U_std',
                                     'IAT_U_10p', 'IAT_U_20p', 'IAT_U_30p', 'IAT_U_40p', 'IAT_U_50p', 'IAT_U_60p', 'IAT_U_70p', 'IAT_U_80p', 'IAT_U_90p',
                                     'IAT_U_F25_avg', 'IAT_U_F50_avg', 'IAT_U_L25_avg', 'IAT_U_L50_avg'

                                     ])

        # read the pcap from given path
        pcap = rdpcap(str(path)+'-1.pcap')

        pcaplen = len(pcap)
        pcaptime_t = int(pcap[pcaplen-1].time - pcap[0].time)
        print(pcaptime_t)
        pcaptime_h = math.ceil(pcaptime_t/2)
        print(pcaptime_h)
        pcaptime_q = math.ceil(pcaptime_t/4)
        print(pcaptime_q)


        # define time-granularity
        granularity = 1; 

        # define threshold values
        dthreshold = 1
        uthreshold = 1

        # initialize start points
        dstart = 0
        ustart = 0

        dinit_pt =0;uinit_pt =0;   

        # define variables
        dpcount, upcount, dpcount100, upcount100 = 0, 0, 0, 0
        dvolume, uvolume, dvolume100, uvolume100 = 0, 0, 0, 0

        dtp, utp, dtp100, utp100 = [], [], [], []
        dtdif, utdif, dtdif100, utdif100 = [], [], [], []
     
        dpktime, upktime, dpktime100, upktime100 = [], [], [], []
        dpktime_fh, upktime_fh, dpktime100_fh, upktime100_fh = [], [], [], []
        dpktime_lh, upktime_lh, dpktime100_lh, upktime100_lh = [], [], [], []
        dpktime_fq, upktime_fq, dpktime100_fq, upktime100_fq = [], [], [], []
        dpktime_lq, upktime_lq, dpktime100_lq, upktime100_lq = [], [], [], []

        diat, uiat, diat100, uiat100 = [], [], [], []
        diat_fh, uiat_fh, diat100_fh, uiat100_fh = [], [], [], []
        diat_lh, uiat_lh, diat100_lh, uiat100_lh = [], [], [], []
        diat_fq, uiat_fq, diat100_fq, uiat100_fq = [], [], [], []
        diat_lq, uiat_lq, diat100_lq, uiat100_lq = [], [], [], []

        dps, ups, dps100, ups100 = [], [], [], []
        dps_fh, ups_fh, dps100_fh, ups100_fh = [], [], [], []
        dps_lh, ups_lh, dps100_lh, ups100_lh = [], [], [], []
        dps_fq, ups_fq, dps100_fq, ups100_fq = [], [], [], []
        dps_lq, ups_lq, dps100_lq, ups100_lq = [], [], [], []


        # iterating through each packet from the pcap file
        for p in pcap:

            # downlink <------------------- (Server - source '10.0.0.251')
            if (p.haslayer(UDP)) and (p[IP].src == '10.0.0.251'):

                # <-------per second based: throughput and packet number
                if (p.time-pcap[0].time) <= dthreshold and (p.time-pcap[0].time) >= dstart:

                    dvolume = dvolume + (int(len(p[IP]))*8)
                    dpcount = dpcount + 1

                else:

                    dtp.append(dvolume/granularity)

                    dpc.append(dpcount)

                    dvolume = 0
                    dpcount = 0

                    dvolume = dvolume + (int(len(p[IP]))*8)
                    dpcount = dpcount + 1

                    dthreshold = dthreshold + 1
                    dstart = dstart + 1

                # <---------- per packet based iat and packet size for entire session
                dtdif = float((p.time-pcap[0].time) - dpktime)

                diat.append(dtdif)

                dpktime = p.time - pcap[0].time

                dps.append(int(len(p[IP])))

                # <--------- per packet based iat and packet size for first quarter session
                if ((p.time - pcap[0].time) <= pcaptime_q):

                    dtdif_fq = float((p.time-pcap[0].time) - dpktime_fq)

                    diat_fq.append(dtdif_fq)

                    dpktime_fq = p.time - pcap[0].time

                    dps_fq.append(int(len(p[IP])))

                # <--------- per packet based iat and packet size for first half session
                if ((p.time - pcap[0].time) <= pcaptime_h):

                    dtdif_fh = float((p.time-pcap[0].time) - dpktime_fh)

                    diat_fh.append(dtdif_fh)

                    dpktime_fh = p.time - pcap[0].time

                    dps_fh.append(int(len(p[IP])))

                # <---------- per packet based iat and packet size for last half session
                if ((p.time - pcap[0].time) > (pcaptime_t - pcaptime_h)) and ((p.time - pcap[0].time) <= pcaptime_t):

                    dtdif_lh = float((p.time-pcap[0].time) - dpktime_lh)

                    diat_lh.append(dtdif_lh)

                    dpktime_lh = p.time - pcap[0].time

                    dps_lh.append(int(len(p[IP])))

                # <---------- per packet based iat and packet size for last quarter session
                if ((p.time - pcap[0].time) > (pcaptime_t - pcaptime_q)) and ((p.time - pcap[0].time) <= pcaptime_t):

                    dtdif_lq = float((p.time-pcap[0].time) - dpktime_lq)

                    diat_lq.append(dtdif_lq)

                    dpktime_lq = p.time - pcap[0].time

                    dps_lq.append(int(len(p[IP])))

            # uplink-------------> (Server - destination '10.0.0.251')
            if (p.haslayer(UDP)) and (p[IP].dst == '10.0.0.251'):

                # ---------> per second based: throughput and packet number
                if (p.time-pcap[0].time) <= uthreshold and (p.time-pcap[0].time) >= ustart:

                    uvolume = uvolume + (int(len(p[IP]))*8)
                    upcount = upcount + 1

                else:
                    utp.append(uvolume/granularity)

                    upc.append(upcount)

                    uvolume = 0
                    upcount = 0

                    uvolume = uvolume + (int(len(p[IP]))*8)
                    upcount = upcount + 1

                    uthreshold = uthreshold + 1
                    ustart = ustart + 1

                # ---------> per packet based iat and packet size for entire session
                utdif = float((p.time - pcap[0].time) - upktime)

                uiat.append(utdif)

                upktime = p.time - pcap[0].time

                ups.append(int(len(p[IP])))

                # ---------> per packet based iat and packet size for first quarter session
                if ((p.time - pcap[0].time) <= pcaptime_q):

                    fq = p.time - pcap[0].time

                    utdif_fq = float((p.time-pcap[0].time) - upktime_fq)

                    uiat_fq.append(utdif_fq)

                    upktime_fq = p.time - pcap[0].time

                    ups_fq.append(int(len(p[IP])))

                # ---------> per packet based iat and packet size for first half session
                if ((p.time - pcap[0].time) <= pcaptime_h):

                    fh = p.time - pcap[0].time

                    utdif_fh = float((p.time-pcap[0].time) - upktime_fh)

                    uiat_fh.append(utdif_fh)

                    upktime_fh = p.time - pcap[0].time

                    ups_fh.append(int(len(p[IP])))

                # ---------> per packet based iat and packet size for last half session
                if ((p.time - pcap[0].time) > (pcaptime_t - pcaptime_h)) and ((p.time - pcap[0].time) <= pcaptime_t):

                    lh = p.time - pcap[0].time

                    utdif_lh = float((p.time-pcap[0].time) - upktime_lh)

                    uiat_lh.append(utdif_lh)

                    upktime_lh = p.time - pcap[0].time

                    ups_lh.append(int(len(p[IP])))

                # ---------> per packet based iat and packet size for last quarter session
                if ((p.time - pcap[0].time) > (pcaptime_t - pcaptime_q)) and ((p.time - pcap[0].time) <= pcaptime_t):

                    lq = p.time - pcap[0].time

                    utdif_lq = float((p.time-pcap[0].time) - upktime_lq)

                    uiat_lq.append(utdif_lq)

                    upktime_lq = p.time - pcap[0].time

                    ups_lq.append(int(len(p[IP])))

        # define the session lengths for throughput and packet number QoS features
        l = len(dtp)
        l1 = math.ceil(len(dtp)/2)
        l2 = math.ceil(len(dtp)/4)
        lu = len(utp)
        lu1 = math.ceil(len(utp)/2)
        lu2 = math.ceil(len(utp)/4)

        print(l)
        print(l1)
        print(l2)

        k = len(dpc)
        k1 = math.ceil(len(dpc)/2)
        k2 = math.ceil(len(dpc)/4)

        print(k)
        print(k1)
        print(k2)

        ku = len(upc)
        ku1 = math.ceil(len(upc)/2)
        ku2 = math.ceil(len(upc)/4)

        print(fq)
        print(fh)
        print(lq)
        print(lh)
        print(pcaptime_t - pcaptime_q)
        print(pcaptime_t - pcaptime_h)

        # insert QoS values for each QoS features

        data.loc[0] = [np.mean(dtp),  np.max(dtp),  np.min(dtp),  np.median(dtp),  np.std(dtp),
                       np.percentile(dtp, 10),  np.percentile(dtp, 20),  np.percentile(dtp, 30),  np.percentile(dtp, 40),  np.percentile(
                           dtp, 50),  np.percentile(dtp, 60),  np.percentile(dtp, 70),  np.percentile(dtp, 80),  np.percentile(dtp, 90),
                       np.mean((dtp[0:l2])), np.mean((dtp[0:l1])), np.mean(
                           (dtp[l-l2:l])), np.mean((dtp[l-l1:l])),

                       np.mean(utp),  np.max(utp),  np.min(
                           utp),  np.median(utp),  np.std(utp),
                       np.percentile(utp, 10),  np.percentile(utp, 20),  np.percentile(utp, 30),  np.percentile(utp, 40),  np.percentile(
                           utp, 50),  np.percentile(utp, 60),  np.percentile(utp, 70),  np.percentile(utp, 80),  np.percentile(utp, 90),
                       np.mean((utp[0:lu2])), np.mean((utp[0:lu1])), np.mean(
                           (utp[lu-lu2:lu])), np.mean((utp[lu-lu1:lu])),

                       np.sum(dpc), np.mean(dpc),  np.max(dpc),  np.min(
                           dpc),  np.median(dpc),  np.std(dpc),
                       np.percentile(dpc, 10),  np.percentile(dpc, 20),  np.percentile(dpc, 30),  np.percentile(dpc, 40),  np.percentile(
                           dpc, 50),  np.percentile(dpc, 60),  np.percentile(dpc, 70),  np.percentile(dpc, 80),  np.percentile(dpc, 90),
                       np.sum((dpc[0:k2])), np.sum((dpc[0:k1])), np.sum(
                           (dpc[k-k2:k])), np.sum((dpc[k-k1:k])),

                       np.sum(upc), np.mean(upc),  np.max(upc),  np.min(
                           upc),  np.median(upc),  np.std(upc),
                       np.percentile(upc, 10),  np.percentile(upc, 20),  np.percentile(upc, 30),  np.percentile(upc, 40),  np.percentile(
                           upc, 50),  np.percentile(upc, 60),  np.percentile(upc, 70),  np.percentile(upc, 80),  np.percentile(upc, 90),
                       np.sum((upc[0:ku2])), np.sum((upc[0:ku1])), np.sum(
                           (upc[ku-ku2:ku])), np.sum((upc[ku-ku1:ku])),


                       np.mean(dps),  np.max(dps),  np.min(
                           dps),  np.median(dps),  np.std(dps),
                       np.percentile(dps, 10),  np.percentile(dps, 20),  np.percentile(dps, 30),  np.percentile(dps, 40),  np.percentile(
                           dps, 50),  np.percentile(dps, 60),  np.percentile(dps, 70),  np.percentile(dps, 80),  np.percentile(dps, 90),
                       np.mean(dps_fq), np.mean(dps_fh), np.mean(
                           dps_lq), np.mean(dps_lh),


                       np.mean(ups),  np.max(ups),  np.min(
                           ups),  np.median(ups),  np.std(ups),
                       np.percentile(ups, 10),  np.percentile(ups, 20),  np.percentile(ups, 30),  np.percentile(ups, 40),  np.percentile(
                           ups, 50),  np.percentile(ups, 60),  np.percentile(ups, 70),  np.percentile(ups, 80),  np.percentile(ups, 90),
                       np.mean(ups_fq), np.mean(ups_fh), np.mean(
                           ups_lq), np.mean(ups_lh),

                       np.mean(diat),  np.max(diat),  np.min(
                           diat),  np.median(diat),  np.std(diat),
                       np.percentile(diat, 10),  np.percentile(diat, 20),  np.percentile(diat, 30),  np.percentile(diat, 40),  np.percentile(
                           diat, 50),  np.percentile(diat, 60),  np.percentile(diat, 70),  np.percentile(diat, 80),  np.percentile(diat, 90),
                       np.mean(diat_fq), np.mean(diat_fh), np.mean(
                           diat_lq), np.mean(diat_lh),


                       np.mean(uiat),  np.max(uiat),  np.min(
                           uiat),  np.median(uiat),  np.std(uiat),
                       np.percentile(uiat, 10),  np.percentile(uiat, 20),  np.percentile(uiat, 30),  np.percentile(uiat, 40),  np.percentile(
                           uiat, 50),  np.percentile(uiat, 60),  np.percentile(uiat, 70),  np.percentile(uiat, 80),  np.percentile(uiat, 90),
                       np.mean(uiat_fq), np.mean(uiat_fh), np.mean(
                           uiat_lq), np.mean(uiat_lh)



                       ]

        export_csv = data.to_csv(str(path)+'-pcap.csv')
        print('export csv ok')

    except:
         print(path)
         pass


#### Configuration over 5G traces

In [ ]:
iteration = 1

mode = ['5g']

net5 = ['static-1', 'static-2', 'static-3',
        'static-4', 'static-5']  # mobility for 5g

tp_type = ['quic']
ap_type = ['http-3_p']
sv_type = ['WSGI_caddy']


video_id = ['v2']  # ['v2']
tl_scheme = ['12x4', '8x4']
vp_traces = 5  # 48
vp_error = [0, 50, 100]
thread = [1]
buffer = [2, 4, 6]
t_segment = [60]
algo = [0, 1]  # adaptation algorithm  FD =0 , FDB = 1

bg_traffic = ['no']

host = [1]

# todo- make nested for loop shorter in an alternative way
for it in range(iteration):
    for i in mode:
        if i == '5g':
           for j in net5:
               for k in tp_type:
                   for l in ap_type:
                       for m in sv_type:
                           for n in video_id:
                               for o in tl_scheme:
                                   for p in range(3, vp_traces+1):
                                        delay = 5
                                        for q in vp_error:
                                            for r in thread:
                                                for s in buffer:
                                                    for t in t_segment:
                                                        for u in algo:
                                                            for v in bg_traffic:
                                                                for w in host:
                                                                    path1 = str(i)+'/'+str(j)+'/'+str(k)+'/'+str(l)+'/'+str(m)+'/'+str(n)+'/'+str(
                                                                        o)+'/'+'VP-trace-'+str(p)+'/'+str(v)+'-bg_traffic'+'/'+'Exp'+str(it+1)+'/'

                                                                    path2 = 'host-'+str(w)+'_ts-'+str(t)+'_thd-'+str(r)+'_vpe-'+str(
                                                                        q)+'_algo-'+str(u)+'_bft-'+str(s)+'_delay-'+str(delay)+'/'
                                                                    filename = 'host-'+str(w)+'_ts-'+str(t)+'_thd-'+str(r)+'_vpe-'+str(
                                                                        q)+'_algo-'+str(u)+'_bft-'+str(s)+'_delay-'+str(delay)
                                                                    path = '/media/tariq/Data-House/vrexp/' + \
                                                                        str(path1)+str(path2) + \
                                                                        str(filename)
                                                                    #print(path)
                                                                    #pcap_filter(path)
                                                                    pcap_processing(path)


#### Configuration over 4G traces

In [ ]:
iteration = 1

mode = ['4g']

net4 = ['train_new_1', 'static_new_1',
        'car_new_1', 'pedestrian_new_1', 'bus_new_1']

tp_type = ['quic']
ap_type = ['http-3_p']
sv_type = ['WSGI_caddy']


video_id = ['v2']  # ['v2']
tl_scheme = ['12x4', '8x4']
vp_traces = 5  # 48
vp_error = [0, 50, 100]
thread = [1]
buffer = [2, 4, 6]
t_segment = [60]
algo = [0, 1]  # adaptation algorithm  FD =0 , FDB = 1

bg_traffic = ['no']

host = [1]

# todo- make nested for loop shorter in an alternative way
for it in range(iteration):
    for i in mode:
        if i == '4g':
           for j in net4:
               for k in tp_type:
                   for l in ap_type:
                       for m in sv_type:
                           for n in video_id:
                               for o in tl_scheme:
                                   for p in range(1, vp_traces+1):
                                        delay = 11
                                        for q in vp_error:
                                            for r in thread:
                                                for s in buffer:
                                                    for t in t_segment:
                                                        for u in algo:
                                                            for v in bg_traffic:
                                                                for w in host:
                                                                    path1 = str(i)+'/'+str(j)+'/'+str(k)+'/'+str(l)+'/'+str(m)+'/'+str(n)+'/'+str(
                                                                        o)+'/'+'VP-trace-'+str(p)+'/'+str(v)+'-bg_traffic'+'/'+'Exp'+str(it+1)+'/'

                                                                    path2 = 'host-'+str(w)+'_ts-'+str(t)+'_thd-'+str(r)+'_vpe-'+str(
                                                                        q)+'_algo-'+str(u)+'_bft-'+str(s)+'_delay-'+str(delay)+'/'
                                                                    filename = 'host-'+str(w)+'_ts-'+str(t)+'_thd-'+str(r)+'_vpe-'+str(
                                                                        q)+'_algo-'+str(u)+'_bft-'+str(s)+'_delay-'+str(delay)
                                                                    path = '/media/tariq/Data-House/vrexp/' + \
                                                                        str(path1)+str(path2) + \
                                                                        str(filename)
                                                                    #pcap_filter(path)
                                                                    pcap_processing(path)
